# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: ** Se baseando somente na pergunta, creio que esse é um problema de classificação binária visto que devo atingir tal objetivo criando um classificador que dado as variáveis de um determinado estudante irá classificar o mesmo como alguém que precisa ou não de intervenção. Por isso, seria um classificador binário, já que temos apenas duas classes (Intervir ou Não intervir).  

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [1]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [2]:
# TODO: Calcule o número de estudante
n_students = student_data.shape[0]

# TODO: Calcule o número de atributos
n_features = student_data.shape[1] - 1

# TODO: Calcule o número de alunos aprovados
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calcule o número de alunos reprovados
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calcule a taxa de graduação
grad_rate = (n_passed/float(n_students))*100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [3]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head(5)

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [7]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [8]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.cross_validation import train_test_split

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test/float(X_all.shape[0]), random_state=11)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

1. __Decision Tree__: Esse é um modelo muito bom para tomada de decisão, visto que o mesmo é, basicamente, um compilado de diversas condições lógicas (If...Then..Else) que encadeadas ajudam o usuário a solucionar seu problema. 
    * __Aplicação__: Imagine que temos dados de pacientes de um hospital e queremos priorizar o atendimento aos indivíduos mais necessitados. Podemos fazer isso usando atributos como idade, pressão sanguínea, peso, doênças na familia, etc.   
    * __Vantagens__: - É um modelo de fácil explicação, mesmo para um leigo, já que ao fim do treinamento poderemos observar a árvore construída e interpretar todos os caminhos da mesma. Além do mais, consegue lidar com variáveis categóricas e numéricas. Ambos os tipos se encontram presentes no nossos dados.
    * __Desvantagens__: É um modelo que pode se tornar instável devido a pequenas variações nos dados resultando em uma árvore completamente diferente da ideal. Ademais, a árvore pode ficar enviesada a predizer uma classe em detrimento de outra, caso tenhamos um dataset muito desbalanceado.  
    * __Por que é um bom modelo pra esse problema?__ O nosso objetivo é identificar se o aluno precisa ou não de itervenção e, acredito que justificar o porquê dessa decisão para os diretores da escola também seja importante. Usando uma decision tree, será mais fácil justificar nossa decisão, visto que para isso basta percorrer o caminho da árvore que levou o modelo a sua conclusão. Além disso, acredito que as variáveis do dataset apresentem alguma construção lógica que tenha relação com a classificação e sabendo que o objetivo desse modelo é encontrar tais construções, então penso que o mesmo seja um bom candidato. E como nosso dataset não apresenta um desbalanceamento de classes significativo, então isso diminui as chances da árvore se tornar enviesada para uma classe.
    
2. __KNN__: Esse é um dos modelos de machine learning mais simples e intuitivo de se entender, pois ele se baseia na ideia de classificar cada  ponto dos dados usando a classificação dos pontos mais próximos. O que é conhecido como __aprendizado baseado em instâncias__.
    * __Aplicação__: É um algoritmo que pode ser usado para predizer o preço de um imóvel baseado nos preços dos imóveis que se encontram geograficamente próximos ao mesmo.
    * __Vantagens__: O tempo de treinamento é constante, visto que não precisamos contruir um modelo propriamente dito. Os nossos dados são o modelo em si. Tal característica é conhecida como __aprendizagem preguiçosa__.
    * __Desvantagens__: Precisamos de memória suficiente para armazenar os dados, pois toda classificação feita é baseado nos mesmos. No nosso caso, isso não é problema, pois estamos lidando com um dataset pequeno.
    * __Por que é um bom modelo pra esse problema?__ _KNN_ é um algoritmo de aprendizado baseado em instâncias. Em outras palvras, ele procura os dados mais próximos/similares aquele ponto para classificá-lo. Pensando dessa forma, acredito que ele é um bom modelo pra essa situação, pois penso que os dois conjuntos de alunos (Precisam de intervenção e não precisam) apresentam um determinado padrão de características próprias. Ou seja, os alunos que precisam de intervenção são similares/próximos uns aos outros. O mesmo deve ocorrer com os que não precisam. Dessa forma, podemos usar a premissa do _KNN_ a fim de classificar um aluno comparando com os alunos de cada um dos conjuntos e verificar quais são aqueles que são mais similares ou estão mais próximos a ele. Além do mais, podemos seguir o  [fluxograma](http://scikit-learn.org/stable/_static/ml_map.png) disponibilizado pelo scikit-learn e verificar que as caracteristicas desse problema (Classes conhecidas, dados classificados e menos de 100 mil linhas de dados) podem nos levar ao _KNN_.
    
3. __Support Vector Machines__: Esse é um modelo um pouco mais complexo do que os anteriores. Esse algoritmo procura encontrar o melhor hiperplano que discrimina os dados.
    * __Aplicação__: Pode ser aplicado pra classificar partes de uma imagem como sendo de uma face ou não. No caso, poderia ter como base de treinamento um vetor de características de cada pixel de uma imagem, no qual cada pixel seria classificado como sendo uma parte da face ou não. 
    * __Vantagens__: Sabe lidar bem com uma grande quantidade de atributos, até mesmo em casos em que a quantidade dos mesmos é maior que a quantidade de dados. 
    * __Desvantagens__: Não prover diretamente uma estimação de probabilidades para as classes. Tal probabilidade pode ser interpretada como quão certo o modelo está sobre a classificação retornada. Alguns modelos não apresentam tal probabilidade. O _SVM_ é um deles, porém existe algumas formas de contornar isso. A biblioteca do scikit-learn, por exemplo, o faz usando um método conhecido como _[Platt scaling](https://en.wikipedia.org/wiki/Platt_scaling)_ que, basicamente, usa regressão logistica aplicada ao resultado do classificador a fim de retornar o grau de certeza da classificação. Contudo, tal método não é sempre eficiente e pode ser muito custoso quando aplicado em datasets grandes como é ressaltado nesta [seção](http://scikit-learn.org/stable/modules/svm.html#scores-and-probabilities) da documentação do scikit-learn.
    * __Por que é um bom modelo pra esse problema?__ Temos um dataset com uma grande quantidade de atributos (Trinta no total) e com uma variável-alvo binária. Sabemos que uma das vantagens do _SVM_ é apresentar uma boa performance nesse tipo de cenário, por isso acredito que ele deve ser um bom candidato. A fim de melhor embasar minha escolha, novamente usei o [fluxograma](http://scikit-learn.org/stable/_static/ml_map.png) e pude constatar que o _SVM_ é um dos modelos que se encontra no caminho de caracteristicas desse problema (Classes conhecidas, dados classificados e menos de 100 mil linhas de dados).
    
Fontes:
* [Scikit ML map](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)


* [SVM Scikit API](http://scikit-learn.org/stable/modules/svm.html#svm)
* [KNN Scikit API](http://scikit-learn.org/stable/modules/neighbors.html)
* [Decision Tree Scikit API](http://scikit-learn.org/stable/modules/tree.html)


* [Decision Tree application](https://pdfs.semanticscholar.org/52a5/c35d9ade36348d8688f20d1de3d9d1cde77c.pdf)
* [SVM application](https://data-flair.training/blogs/applications-of-svm/) - Primeiro exemplo.
* [KNN application](https://github.com/soutik/Boston-Housing-dataset-exploration-with-KNN/blob/master/Boston-Housing-KNN.ipynb)

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [11]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

# TODO: Inicialize os três modelos
clf_A = svm.SVC(random_state=10)
clf_B = KNeighborsClassifier()
clf_C = tree.DecisionTreeClassifier(random_state=10)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in [clf_A, clf_B, clf_C]:
    train_predict(clf, X_train_100, y_train_100, X_test, y_test)
    print
    train_predict(clf, X_train_200, y_train_200, X_test, y_test)
    print 
    train_predict(clf, X_train_300, y_train_300, X_test, y_test)
    print "#########################"

Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0024 segundos
As previsões foram feitas em 0.0014 segundos.
Pontuação F1 para o conjunto de treino: 0.8684.
As previsões foram feitas em 0.0011 segundos.
Pontuação F1 para o conjunto de teste: 0.7945.

Treinando um SVC com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0054 segundos
As previsões foram feitas em 0.0039 segundos.
Pontuação F1 para o conjunto de treino: 0.8758.
As previsões foram feitas em 0.0020 segundos.
Pontuação F1 para o conjunto de teste: 0.8235.

Treinando um SVC com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0114 segundos
As previsões foram feitas em 0.0085 segundos.
Pontuação F1 para o conjunto de treino: 0.8596.
As previsões foram feitas em 0.0028 segundos.
Pontuação F1 para o conjunto de teste: 0.8344.
#########################
Treinando um KNeighborsClassifier com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0009 segundos
As previsões foram fei

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - SVM**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |         0.0024       |  0.0011                    |    0.8684                  |  0.7945                     |
| 200                                |       0.0054         |      0.0020                 |           0.8758           | 0.8235                    |
| 300                                |        0.0114        |             0.0028          |           0.8596           | 0.8344      


** Classificador 2 - KNN**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |   0.0009             |    0.0016                   |  0.8322                   | 0.8028                     |
| 200                                |      0.0009        |  0.0024                     |  0.8718                    | 0.7972                    |
| 300                                |       0.0011        |  0.0033                    |   0.8623                   |        0.8148       |

** Classificador 3 - Decision Tree**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |0.0012                | 0.0002                      | 1.0000                    | 0.7077                     |
| 200                                |0.0019                | 0.0003                      |  1.0000                    |0.6504                     |
| 300                                |0.0029                | 0.0003                      | 1.0000                     | 0.7460                     |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: ** Após avaliar a tabela com os resultados, o modelo escolhido é o __SVM__, pois, apesar do mesmo apresentar um tempo de treinamento superior comparado aos outros modelos, ele tem um F1 nos dados de teste superior aos outros dois modelos. Além disso, observando os resultados, vemos que o modelo de __decision tree__ sofreu overvitting (F1 igual à 1 no treinamento) e, por isso, não consegue generalizar bem (F1 baixo nos dados de testes), ao passo que o __knn__, apresentou uma performance quase tão boa quanto o __SVM__, obtendo uma média de F1 nos testes de __0.8049__. Contudo, __SVM__ foi um pouco superior, obtendo uma média de F1 nos testes de __0.8174__.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: ** Todo o processo de decidir qual o melhor modelo a ser aplicado começa sempre com a divisão dos dados em __treino__ e __teste__, onde usamos o treino para construir o modelo e o teste para avaliá-lo. A ideia aqui é que o modelo aprenda a melhor forma de classificar os alunos a partir de exemplos que já se encontram classificados. Uma vez que o modelo tenha passado por essa fase de __treinamento__, podemos aplicá-lo nos dados de teste e avaliar sua performance comparando a classificação sugerida pelo modelo e a classificação real (**Fase de teste**). Existe diversas métricas para mensurar a performance do modelo. Nesse caso, nós estamos considerando uma métrica de avaliação conhecida como __F-Measure__. Quanto maior o valor da mesma, melhor nosso modelo, e maiores são as chances do mesmo acertar se um determinado aluno precisa ou não de intervenção. Com isso em mente e observando a coluna de pontuação nos dados de teste, vemos que o modelo __SVM__ é o que apresentou um melhor desempenho. Além disso, é importante notar, também, que ele teve um tempo de treinamento significamente superior aos outros, porém como estamos lidando com poucos dados, isso não chega a ser um problema. Contudo, caso essa base tenha um aumento substancial e o modelo precise ser retreinado, então isso pode vir a ser um tópico de discussão no futuro.
De toda forma, é bom ter um entendimento básico de como nosso modelo funciona na prática. No nosso caso, imagine que temos dois conjuntos de pontos: Alunos que precisam de Intervenção e Alunos que não precisam. Tais pontos estão distribuidos no espaço de alguma forma e para cada um deles, nós temos um conjunto de atributos relacionado (Sexo, idade, Profissão do Pai, etc). Cada atributo tem uma influência sobre a posição dos pontos no espaço. Simplificadamente falando, o SVM procura encontrar a melhor "linha" que separa nossos dados. Tal "ĺinha" é conhecida como hiperplano. A fim de encontrar essa "linha", o SVM pode fazer o mapeamento dos nossos pontos, quando necessário, para um espaço de maior dimensão de forma que os pontos possam ser melhor separados. Em outras palavras, o SVM apenas precisa descobrir a melhor linha/hiperplano que classifica nossos dados. Feito isso, ele usa tal linha pra classificar novas instâncias observando em qual lado do hiperplano as mesmas se encontram. Abaixo segue uma ilustração para melhor entendimento.

<p> Imagine que temos o problema de classificar um ponto como azul ou vermelho e, para isso, nós temos o conjunto de dados abaixo. Uma estratégia seria encontrar uma forma de separar tal conjunto de dados, onde de um lado temos apenas azuis e do outro vermelho. Assim sendo, quando chegar um novo ponto para classificar, só precisamos observar em qual lado ele ficou. Podemos fazer uma analogia com o problema original e supor que os pontos azuis e vermelhos representam alunos que não precisam de intervenção e alunos que precisam, respectivamente.</p>
<img src="svm1.png" title="Title text" />

<p>Uma <i>linha</i> aparenta ser uma boa maneira de fazer a separação. Como dito anteriormente, o termo correto para essa linha é <b>hiperplano</b>. A figura abaixo apresenta três diferentes hiperplanos que separam os dados, porém o amarelo é o melhor, pois o __SVM__ procura encontrar o hiperplano que maximiza a distância entre as duas classes de pontos. Perceba que o amarelo é o único que apresenta uma distância segura entre os dois tipos de pontos. Ao passo que, o lilás está bastante próximo do vermelho, e o rosa do azul, o que pode causar erros na classificação.</p>
<img src="svm2.png" title="Title text" />

<p>Abaixo vemos o melhor hiperplano dividindo nossos dados. Todo o processo que nos levou a encontrar o hiperplano amarelo é a fase de treinamento de modelo como foi mencionado anteriormente. De agora em diante, nós podemos usar tal hiperplano pra classificar um ponto desconhecido baseado em qual lado do hiperplano o mesmo se encontra. Caso nós tenhamos um diferente conjunto de pontos e que já estejam classificados como vermelho ou azul, então nós podemos usá-los  para saber o quão bom é nosso hiperplano, bastando apenas comparar a classificação real do ponto com aquela que o hiperplano diz (Fase de teste). Quanto mais as duas classificações sejam iguais, mais segurança nós temos que escolhemos um bom hiperplano e, consequentemente, um bom modelo para nosso problema. </p>
<img src="svm3.png" title="Title text" />

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [12]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {'kernel': ["linear", "poly", "rbf", "sigmoid"], 
              'C': [1, 5, 10]}

# TODO: Inicialize o classificador
clf = svm.SVC()

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

As previsões foram feitas em 0.0035 segundos.
O modelo calibrado tem F1 de 0.8000 no conjunto de treinamento.
As previsões foram feitas em 0.0012 segundos.
O modelo calibrado tem F1 de 0.8125 no conjunto de teste.


In [13]:
print "Melhores parâmetros: ", grid_obj.best_params_

Melhores parâmetros:  {'kernel': 'sigmoid', 'C': 1}


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

O modelo final apresentou uma pontuação de _0.8000_ e _0.8125_ nos conjuntos de treino e teste respectivamente. Ele apresentou uma leve queda de performance quando comparado com o modelo não calibrado, já que esse conseguiu um F1 de _0.8596_ e _0.8344_ no treino e teste respectivamente. E considerando 300 pontos para treinar. 

**Resposta: **

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.